# Research notes

### General notes

* Two regimes in time: annealing, and then settled
* Two regimes in setup: Egalitarian (Free knowledge) and dictator (Centralized knowledge)

---
### Model modification ideas

#### ⬛ **Major idea 1:** Groups have many dimensions

* Groups have the same number of preference dimensions as individuals
* Groups have fewer dimensions than individuals
 * Individuals need to balance their fulfillment with groups? How do they choose?
  * **Choice rule 1:** Individuals focus on the dimension which is furthest from any existing group ("I join this group to fulfill something I can't fulfill alone")
  * **Choice rule 2:** Individuals join multiple groups until all their dimensions are met
* Groups have more dimensions than individuals??
 * The groups' identity changes as the members who make it up change
 * Absent any within-group dynamics, why would this be any different than having separate groups?

#### ⬛ **Major idea 2:** Multiple group membership

* Individuals must join X groups, where X > 1
* Individuals join as many groups as fulfills them. This may need to involve some counterbalancing mechanic where individuals suffer a penalty for joining too many groups
 * Fulfillment : closest for each dimension
 * Fulfillment : closets for important dimensions

#### ⬛ **Major idea 3:** Groups splitting and joining

Easier goal would be to set up the rules explicitly and see how individual choices affects group ecology. 

* Encoding rules for groups splitting and fusing explicitly
  * Splitting because of size (i.e. group > 300 so it splits)
  * Splitting because of difference? (i.e. group's distribution has two modes, so it splits)
  * Splitting because of an optimization parameter (i.e. if a subset of the group would have decreased dissimilarity, it splits)

Harder goal would be to set up rules such that groups split and join dynamically. 

* Create set of rules that defines the substance/scope of group identity, and less about parsing out "least bad alternatives"

#### ⬛ **Major idea 4:** Bounded confidence

* Bounded confidence? Overlapping latitudes of acceptance/rejection? Ideas from discussion with Harry, terms from Jiin
 * Various inspirations from polarization psych analogous to network stuff

---

#### Other ideas

* Pure random model
* Let group hold its value before dying, from Jordan and Ismael. 
 * PROS: Gets at a sense of 'history'
 * CONS: It may not contribute to novel community growth?
* Heterogeneous accessibility / central media
 * Try to replicate the crash-and-burn dynamics of a central group actor
* Individuals are in a network (what would this mean?), or groups are in a network (what would this mean?)
* Mean voting for groups. Other mechanics/functions for group's preference value
* Circular preference dimension
* Write out the abstract/mathy form of this model. Function for choice, function for group aggregation of value 

### Model checks

* Test if it matters that individual values are discrete or continuous. Int vs float

#### Tossed

* Initialize with "closest group", from Sabina. Now with the model fixed, this will likely just be immediately stable. (DONE)

-----

### General dataset comparison ideas

* Crash-and-burn dynamics of platform chasing
* Within-person group changing dynamics
* Catching an ‘invisible exodus’ of people from one subreddit, and checking if they move to another subreddit? If the other sub survives?


### Specific dataset comparison ideas

* Take the times between group-switching for a particular person. Compare them to the times between subreddit-switching posting for a particular person? (Here posts==time)

--- 

# Setup functions

This should try its best to separate the following:
* Editing individual's relationships to groups
* Groups updating their values
* Any effects on public information



Class-version where you extend the graph class to include the individual function and group function. Good for instantiating many versions of the simulation. 

## Universal Functions

In [ ]:
import networkx as nx
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import time
import pandas as pd
import seaborn as sns
sns.set_theme(style="darkgrid")
import functools
from copy import deepcopy

In [ ]:
def doNothing(): #For debugging purposes when a function needs to be passed in
  pass

clamp = lambda n, minn, maxn: max(min(maxn, n), minn)

def grabMeans(netw,grouplist):
  return {i: netw.nodes[i]['mean'] for i in grouplist}

def grabGroupsize(netw,grouplist):
  return {group: len(list(netw.neighbors(group))) for group in grouplist}

def distMatrix1D(m): #Coded this up quickly because I'm too lazy to import scipy
  m = np.array(m)
  dist = np.tile(m,(len(m),1))
  return np.abs(dist - dist.T)

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

Next, I want to implement a tracker to keep track of the following:

* Mean difference between indiv and group valence (as a mean of that difference, per group, over time)
* Variance of differences between indiv and group valence (as calculated from the distribution of valences, per group, over time)

In [ ]:
class GroupshiftSim():
    
#===============================================================================
# Intializing
#===============================================================================

  def __init__(self, groupshift_function, indivchoice_function, lowvalence = 0, highvalence =100, s = 1, inittype='random', save_timesteps = []):
    self.lowvalence = lowvalence
    self.highvalence = highvalence
    self.s = s

    self.savemeans = []
    self.savesizes = []
    self.save_timesteps = [0] + save_timesteps
    self.saved_graphs = {}

    self.groupShift = groupshift_function
    self.indivChoice = indivchoice_function
    #Valence range?
    #Valence dimensions?
    #Toroid?
    #Resettype?
    if inittype not in ['random', 'sorted']:
      raise Exception("Invalid type of graph connectivity initialization for this groupshift simulation.")

  def initializeNodes(self, num_nodes, num_groups, inittype): 
    self.num_nodes = num_nodes
    self.num_groups = num_groups
    self.grouplist = ['Group%i'%i for i in range(num_groups)]
    self.nodelist = list(range(num_nodes))
    
    #Creating graph
    self.g = nx.Graph()
    if inittype == 'random':
      nodes_with_valence = [(i,{'valence':np.random.randint(self.lowvalence,self.highvalence, size=self.s)}) for i in self.nodelist]
      self.g.add_nodes_from( nodes_with_valence )   #Add nodes with attribute 'valence': (R,G,B)
      self.g.add_nodes_from( self.grouplist ) #Add grouplist nodes    
      self.g.add_edges_from( [(i,np.random.choice(self.grouplist)) for i in self.nodelist] )  #Add edges randomly between nodes and grouplist nodes
    elif inittype == 'sorted':
      self.g.add_nodes_from( self.grouplist )
      self.g.add_nodes_from( self.nodelist )
      edgelist = [(i,np.random.choice(self.grouplist)) for i in self.nodelist]
      self.g.add_edges_from( edgelist )
      sorted_nodes, sorted_groups = zip(*sorted(edgelist, key=lambda kv:(kv[1], kv[0])))
      sorted_valences = sorted( [np.random.randint(self.lowvalence,self.highvalence, size=self.s) for i in range(self.num_nodes)] )
      valence_dict = {sorted_nodes[i]: sorted_valences[i] for i in range(len(sorted_nodes)) }
      nx.set_node_attributes(self.g, valence_dict, 'valence')
    
#===============================================================================
# Running simulations, saving data
#===============================================================================

  def runSimulation(self, timesteps, **kwargs):
    self.timesteps = timesteps
    self.save_timesteps.append(timesteps-1)
    for t in tqdm(range(self.timesteps)):
      #Save out the graph data
      if t in self.save_timesteps:
        self.saved_graphs[t] = deepcopy(self.g)

      #Perform group shift function
      self.groupShift(self)

      #Save out the statistical data
      thesemeans = grabMeans(self.g,self.grouplist)
      thesesizes = grabGroupsize(self.g,self.grouplist)
      self.savemeans.append(thesemeans)
      self.savesizes.append(thesesizes)

      #Perform node selection step function
      self.indivChoice(self, thesemeans, kwargs) #How to parameterize? 

#===============================================================================
# Plotting functions
#===============================================================================

  def plotTimelineMeans(self, legend=True):
    #Grabbing and plotting means over time
    meansdata = {i:[] for i in self.grouplist}
    for m in self.savemeans:
      for group in self.grouplist:
        meansdata[group].append(m[group])

    plt.figure(figsize=(15,10))
    for group in self.grouplist:
      thisthing = np.array(meansdata[group]).T
      plt.plot(thisthing[0], linewidth=1, label=group)

    plt.title("Group means over time, 1D")
    plt.xlabel("Time (t)")
    plt.ylabel("Mean valence")
    plt.ylim(self.lowvalence,self.highvalence)
    if legend: 
      plt.legend()
    plt.show()

  def plotTimelineSizes(self, legend=True):
    #Constructing a tidy dataframe
    to_dataframe = {'group': [], 'timestep': [], 'size': []}
    for i, m in enumerate(self.savesizes):
      for group in self.grouplist:
        to_dataframe['group'].append(group)
        to_dataframe['timestep'].append(i)
        to_dataframe['size'].append(m[group])
      
    modeldf = pd.DataFrame(to_dataframe)
    sns.set(rc={'figure.figsize':(11.7,8.27)})
    sns.lineplot(x='timestep', y='size', hue='group', data = modeldf, linewidth=1, legend=legend).set(title="Group sizes over time")

  def plotIndivDist(self, t='now', overall = True, legend=True):
    if t=='now':
      thisgraph = self.g
    elif type(t) == int:
      if t not in self.saved_graphs:
        raise Exception("The graph at the timestep you requested has not been saved. This timestep is not available.")
      thisgraph = self.saved_graphs[t]      
    else: 
      raise Exception("Invalid input for timeslice parameter for plotting the distribution of individual values.")

    agent_valences = nx.get_node_attributes(thisgraph,'valence')
    agent_sets = {xx:list(thisgraph.neighbors(xx)) for xx in self.grouplist} #Grab the agents associated with each group
    agent_sets_valence = {xx:[agent_valences[i][0] for i in agent_sets[xx]] for xx in self.grouplist} #Grab the valence associated with each agent, by group


    if overall:
      agent_valences_squish = [x[0] for x in agent_valences.values()]
      sns.set(rc={'figure.figsize':(11.7,8.27)})
      sns.displot(agent_valences_squish, bins=15, kde=True).set(title='Distributions of agent opinions, all')
      plt.xlabel('Valence')
      plt.ylabel('Num agents')
    else: 
      data_to_plot = [agent_sets_valence[xx] for xx in self.grouplist]
      sns.displot(data_to_plot, kind='kde', legend=legend).set(title="Distributions of agent opinions by group")

# Running

## Common functions

Certain functions, like groups calculating their means, will occur across multiple models, but they aren't necessarily universal functions in that they are technically parameters of the model. 

In [ ]:
#This function edits a graph in place
def recalcMean(self, netw, groupname, resettype='sust', s=1):
  groupneighs = list(netw.neighbors(groupname))
  if len(groupneighs)>0:
    thismean = np.mean( np.vstack( [netw.nodes[eachnode]['valence'] for eachnode in groupneighs] ), axis=0) #Stack the RGB valences and take the mean
    netw.nodes[groupname]['mean'] = thismean
  else:
    #Set default to zero. This may cause the homogenization
    if resettype == 'zero':
      netw.nodes[groupname]['mean'] = np.zeros(s)
    elif resettype == 'rand':
      #Set default to random. 
      netw.nodes[groupname]['mean'] = np.random.randint(self.lowvalence,self.highvalence,size=s)
    elif resettype == 'sust':
      pass #Do nothing and let the mean be the same as it was

def groupshiftSimpleMean(self, resettype='sust', s=1):
  for i in self.grouplist: 
    recalcMean(self, self.g, i, resettype=resettype, s=s)

## Model 1 Baseline

In [ ]:
def oneNodeOpenSwitch(self, public_info, kwargs):
  #Randomly select a node
  thisnode = np.random.choice(self.nodelist)

  #Calculate distances (for colors)
  dists = {}
  for i in public_info:
    dists[i] = np.linalg.norm(public_info[i] - self.g.nodes[thisnode]['valence'])
  #Get the group with the minimum distance
  mingroup = min(dists,key=dists.get)
  #Replace current group edge
  currentgroup = list(self.g.neighbors(thisnode))[0]
  if currentgroup != mingroup:
    self.g.remove_edge(thisnode,currentgroup)
    self.g.add_edge(thisnode,mingroup)

In [ ]:
testSim = GroupshiftSim(groupshift_function = groupshiftSimpleMean,
                        indivchoice_function = oneNodeOpenSwitch,
                        save_timesteps=list(range(0,10000,1000)))

num_nodes = 1000
num_groups = 10
timesteps = 10000

testSim.initializeNodes(num_nodes, num_groups, 'random')
testSim.runSimulation(timesteps)

In [ ]:
testSim.plotIndivDist(t=0, overall=False)
testSim.plotIndivDist(t=1000, overall=False)
testSim.plotIndivDist(t=4000, overall=False)
testSim.plotIndivDist(overall=False)

In [ ]:
testSim.plotTimelineMeans()
testSim.plotTimelineSizes()

## Model 2 Including error

Inherited from model 1, but agent slightly changes randomly at each change. 

In [ ]:
def oneNodeOpenSwitchWithError(self, public_info, kwargs):
  #Randomly select a node
  thisnode = np.random.choice(self.nodelist)

  #Calculate distances (for colors)
  dists = {}
  for i in public_info:
    dists[i] = np.linalg.norm(public_info[i] - self.g.nodes[thisnode]['valence'])
  #Get the group with the minimum distance
  mingroup = min(dists,key=dists.get)
  #Replace current group edge
  currentgroup = list(self.g.neighbors(thisnode))[0]
  if currentgroup != mingroup:
    self.g.remove_edge(thisnode,currentgroup)
    self.g.add_edge(thisnode,mingroup)
  
  #Add randomly noise to agent's value
  self.g.nodes[thisnode]['valence'] = [clamp(np.random.normal(self.g.nodes[thisnode]['valence'][x],kwargs['e']),self.lowvalence,self.highvalence) for x in range(self.s)]

In [ ]:
testSim = GroupshiftSim(groupshift_function = groupshiftSimpleMean,
                        indivchoice_function = oneNodeOpenSwitchWithError)

num_nodes = 1000
num_groups = 10
timesteps = 10000

testSim.initializeNodes(num_nodes, num_groups, 'random')
testSim.runSimulation(timesteps, e=100) 
# I don't know why, but e has to be pretty big in order for there to be an effect on the groups. 

In [ ]:
testSim.plotTimelineMeans()
testSim.plotTimelineSizes()

## Model 3 Latitude of acceptability

Limiting individual's capacity to choose groups based on their current group

In [ ]:
def model3(num_nodes,num_groups,timesteps, k,plots=False,resettype='zero', s=1):
  grouplist = ['Group%i'%i for i in range(num_groups)]
  #Creating graph
  g = nx.Graph()
  g.add_nodes_from( [(i,{'valence':np.random.randint(0,255,size=s)}) for i in range(num_nodes)] )   #Add nodes with attribute 'valence': (R,G,B)
  g.add_nodes_from( grouplist ) #Add grouplist nodes
  g.add_edges_from( [(i,np.random.choice(grouplist)) for i in range(num_nodes)] )  #Add edges randomly between nodes and grouplist nodes

  #Plot before-and-after, if desired
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()

  savemeans = []
  savesizes = []

  for t in tqdm(range(timesteps)):
    #Recalculate the means
    for i in grouplist:
      meanRecalc(g,i,resettype,s=s)
    #Randomly select a node
    thisnode = np.random.choice(range(num_nodes))

    #Grab statistics
    thesemeans = grabMeans(g,grouplist)
    thesesizes = grabGroupsize(g,grouplist)
    savemeans.append(thesemeans)
    savesizes.append(thesesizes)

    #Calculate distances between each group
    currentgroup = list(g.neighbors(thisnode))[0]
    #Get the k nearest groups
    gdists = {}
    for i in thesemeans:
      gdists[i] = np.linalg.norm(thesemeans[i] - g.nodes[currentgroup]['mean'])
    #Get the groupS (MULTIPLE!) within the latitude of acceptability
    mingroups = sorted(gdists, key=gdists.get)[0:k]

    #Calculate distances against the individual
    dists = {}
    for i in mingroups:
      dists[i] = np.linalg.norm(thesemeans[i] - g.nodes[thisnode]['valence'])
    #Get the group with the minimum distance
    mingroup = min(dists,key=dists.get)
    #Replace current group edge
    if currentgroup != mingroup:
      g.remove_edge(thisnode,currentgroup)
      g.add_edge(thisnode,mingroup)
    
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()
  
  otherdata = {'means': savemeans, 'sizes': savesizes}
  
  return g, otherdata

## Model 4 Simple dictator

In [ ]:
def model4(num_nodes,num_groups,timesteps,plots=False,resettype='zero', s=1):
  grouplist = ['Group%i'%i for i in range(num_groups)]
  #Creating graph
  g = nx.Graph()
  g.add_nodes_from( [(i,{'valence':np.random.randint(0,255,size=s)}) for i in range(num_nodes)] )   #Add nodes with attribute 'valence': (R,G,B)
  g.add_nodes_from( grouplist ) #Add grouplist nodes
  g.add_edges_from( [(i,np.random.choice(grouplist)) for i in range(num_nodes)] )  #Add edges randomly between nodes and grouplist nodes

  #Plot before-and-after, if desired
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()

  savemeans = []
  savesizes = []

  for t in tqdm(range(timesteps)):
    #Recalculate the means
    for i in grouplist:
      meanRecalc(g,i,resettype,s=s)
    #Randomly select a node
    thisnode = np.random.choice(range(num_nodes))

    #Grab statistics
    thesemeans = grabMeans(g,grouplist)
    thesesizes = grabGroupsize(g,grouplist)
    savemeans.append(thesemeans)
    savesizes.append(thesesizes)

    #Calculate distances (for colors)
    dists = {}
    for i in thesemeans:
      dists[i] = np.linalg.norm(thesemeans[i] - g.nodes[0]['valence']) #Calculated against the preference of a single node
    #Get the group with the minimum distance
    mingroup = min(dists,key=dists.get)
    #Replace current group edge
    currentgroup = list(g.neighbors(thisnode))[0]
    if currentgroup != mingroup:
      g.remove_edge(thisnode,currentgroup)
      g.add_edge(thisnode,mingroup)
    
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()
  
  otherdata = {'means': savemeans, 'sizes': savesizes}
  
  return g, otherdata

## Model 5 Modifier: Central Media

In [ ]:
def model5(num_nodes,num_groups,timesteps,plots=False,resettype='zero', s=1):
  grouplist = ['Group%i'%i for i in range(num_groups)]
  #Creating graph
  g = nx.Graph()
  g.add_nodes_from( [(i,{'valence':np.random.randint(0,255,size=s)}) for i in range(num_nodes)] )   #Add nodes with attribute 'valence': (R,G,B)
  g.add_nodes_from( grouplist ) #Add grouplist nodes
  g.add_edges_from( [(i,np.random.choice(grouplist)) for i in range(num_nodes)] )  #Add edges randomly between nodes and grouplist nodes

  #Plot before-and-after, if desired
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()

  savemeans = []
  savesizes = []

  for t in tqdm(range(timesteps)):
    #Recalculate the means
    for i in grouplist:
      meanRecalc(g,i,resettype,s=s)
    #Randomly select a node
    thisnode = np.random.choice(range(num_nodes))

    #Grab statistics
    thesemeans = grabMeans(g,grouplist)
    thesesizes = grabGroupsize(g,grouplist)
    savemeans.append(thesemeans)
    savesizes.append(thesesizes)

    #Calculate distances (for colors)
    dists = {}
    for i in thesemeans:
      d = np.linalg.norm(thesemeans[i] - g.nodes[thisnode]['valence'])
      groupnum = int(''.join(map(lambda c: c if c in '0123456789' else '', i))) #Modify the distances
      dists[i] = d*(float(groupnum+1)/num_groups)
    #Get the group with the minimum distance
    mingroup = min(dists,key=dists.get)
    #Replace current group edge
    currentgroup = list(g.neighbors(thisnode))[0]
    if currentgroup != mingroup:
      g.remove_edge(thisnode,currentgroup)
      g.add_edge(thisnode,mingroup)
    
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()
  
  otherdata = {'means': savemeans, 'sizes': savesizes}
  
  return g, otherdata

## Model 6 Group dictator

In [ ]:
def model6(num_nodes,num_groups,timesteps,plots=False,resettype='zero', s=1):
  grouplist = ['Group%i'%i for i in range(num_groups)]
  #Creating graph
  g = nx.Graph()
  g.add_nodes_from( [(i,{'valence':np.random.randint(0,255,size=s)}) for i in range(num_nodes)] )   #Add nodes with attribute 'valence': (R,G,B)
  g.add_nodes_from( grouplist ) #Add grouplist nodes
  g.add_edges_from( [(i,np.random.choice(grouplist)) for i in range(num_nodes)] )  #Add edges randomly between nodes and grouplist nodes

  #Plot before-and-after, if desired
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()

  savemeans = []
  savesizes = []

  for t in tqdm(range(timesteps)):
    #Recalculate the means
    for i in grouplist:
      meanRecalc(g,i,resettype,s=s)
    #Randomly select a node
    thisnode = np.random.choice(range(num_nodes))

    #Grab statistics
    thesemeans = grabMeans(g,grouplist)
    thesesizes = grabGroupsize(g,grouplist)
    savemeans.append(thesemeans)
    savesizes.append(thesesizes)

    #Randomly select member of group 0
    dictatornode = np.random.choice( list(g.neighbors('Group0')) )
    #Calculate distances (for colors)
    dists = {}
    for i in thesemeans:
      dists[i] = np.linalg.norm(thesemeans[i] - g.nodes[dictatornode]['valence']) #Calculated against the preference of a single node
    #Get the group with the minimum distance
    mingroup = min(dists,key=dists.get)
    #Replace current group edge
    currentgroup = list(g.neighbors(thisnode))[0]
    if currentgroup != mingroup:
      g.remove_edge(thisnode,currentgroup)
      g.add_edge(thisnode,mingroup)
    
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()
  
  otherdata = {'means': savemeans, 'sizes': savesizes}
  
  return g, otherdata

## Model 7 Change rates 1

At every timestep, randomly update the opinion of a single agent. Change group update function to update by rate toward mean. 

In [ ]:
def meanAdjust(netw, groupname, rate, resettype, s=3):
  groupneighs = list(netw.neighbors(groupname))
  if len(groupneighs)>0:
    oldmean = netw.nodes[groupname]['mean']
    truemean = np.mean( np.vstack( [netw.nodes[eachnode]['valence'] for eachnode in groupneighs] ), axis=0) #Stack the RGB valences and take the mean
    if oldmean < truemean: 
      netw.nodes[groupname]['mean'] = oldmean + rate
    elif oldmean > truemean:
      netw.nodes[groupname]['mean'] = oldmean - rate
  else:
    #Set default to zero. This may cause the homogenization
    if resettype == 'zero':
      netw.nodes[groupname]['mean'] = np.zeros(s)
    elif resettype == 'rand':
      #Set default to random. 
      netw.nodes[groupname]['mean'] = np.random.randint(0,255,size=s)
    elif resettype == 'sust':
      pass #Do nothing and let the mean be the same as it was

In [ ]:
def model7(num_nodes,num_groups,timesteps,eta, theta, plots=False,resettype='zero', s=1):
  grouplist = ['Group%i'%i for i in range(num_groups)]
  #Creating graph
  g = nx.Graph()
  g.add_nodes_from( [(i,{'valence':np.random.randint(0,255,size=s)}) for i in range(num_nodes)] )   #Add nodes with attribute 'valence': (R,G,B)
  g.add_nodes_from( grouplist ) #Add grouplist nodes
  g.add_edges_from( [(i,np.random.choice(grouplist)) for i in range(num_nodes)] )  #Add edges randomly between nodes and grouplist nodes

  #Instantiate group means
  for i in grouplist:
    meanRecalc(g,i,resettype,s=s)

  #Plot before-and-after, if desired
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()

  savemeans = []
  savesizes = []

  for t in tqdm(range(timesteps)):
    #Recalculate the means
    for i in grouplist:
      meanAdjust(g,i,theta,resettype,s=s)
    #Randomly select a node
    thisnode = np.random.choice(range(num_nodes))

    #Grab statistics
    thesemeans = grabMeans(g,grouplist)
    thesesizes = grabGroupsize(g,grouplist)
    savemeans.append(thesemeans)
    savesizes.append(thesesizes)

    #Calculate distances (for colors)
    dists = {}
    for i in thesemeans:
      dists[i] = np.linalg.norm(thesemeans[i] - g.nodes[thisnode]['valence'])
    #Get the group with the minimum distance
    mingroup = min(dists,key=dists.get)
    #Replace current group edge
    currentgroup = list(g.neighbors(thisnode))[0]
    if currentgroup != mingroup:
      g.remove_edge(thisnode,currentgroup)
      g.add_edge(thisnode,mingroup)

    #Randomly update one agent's opinion
    shiftnode = np.random.choice(range(num_nodes))
    oldvalence = g.nodes[shiftnode]['valence']
    groupmean = g.nodes[list(g.neighbors(thisnode))[0]]['mean']
    if oldvalence < groupmean:
      g.nodes[shiftnode]['valence'] = oldvalence + eta
    elif oldvalence > groupmean:
      g.nodes[shiftnode]['valence'] = oldvalence - eta
    
  if plots==True:
    pos = nx.spring_layout(g)
    nx.draw(g,pos=pos,node_size=5)
    plt.show()
  
  otherdata = {'means': savemeans, 'sizes': savesizes}
  
  return g, otherdata

## Model 8 Simple suboptimal choices

In [ ]:
def oneNodeOpenSwitchSuboptimal(self, public_info, kwargs):
  #Randomly select a node
  thisnode = np.random.choice(self.nodelist)

  #Calculate distances (for colors)
  dists = {}
  for i in public_info:
    dists[i] = np.linalg.norm(public_info[i] - self.g.nodes[thisnode]['valence'])
  #Get the group with the minimum distance
  dist_order = sorted(dists,key=dists.get)
  #Replace current group edge
  currentgroup = list(self.g.neighbors(thisnode))[0]
  currentgroupindex = dist_order.index(currentgroup)
  if currentgroupindex > 0:
    newgroup = dist_order[currentgroupindex-1]
    self.g.remove_edge(thisnode,currentgroup)
    self.g.add_edge(thisnode,newgroup)

In [ ]:
num_nodes = 2500
num_groups = 50
timesteps = 30000

testSim = GroupshiftSim(groupshift_function = groupshiftSimpleMean,
                        indivchoice_function = oneNodeOpenSwitchSuboptimal,
                        save_timesteps=list(range(0,timesteps,1000)))

testSim.initializeNodes(num_nodes, num_groups, 'random')
testSim.runSimulation(timesteps)

In [ ]:
testSim.plotTimelineMeans(legend=False)
testSim.plotTimelineSizes(legend=False)

In [ ]:
for i in range(0,timesteps,1000):
  testSim.plotIndivDist(t=i, overall=False)
testSim.plotIndivDist(overall=False)

# Misc code pile

In [ ]:
#Plot running average

binsize = 250
plt.figure(figsize=(15,10))

for group in grouplist:
  thisthing = np.array(meansdata[group]).T
  plt.plot(moving_average(thisthing[0], binsize), linewidth=3, label=group)

plt.title("Group means over time, 1D")
plt.xlabel("Time (t)")
plt.ylabel("Mean valence")
plt.ylim(0,255)
plt.legend()
plt.show()